In [ ]:
%matplotlib widget

from matplotlib import pyplot as plt
import os
from themachinethatgoesping.echosounders.simrad import FileSimradRaw,FileSimradRaw_mapped
import themachinethatgoesping.echosounders.simrad as simrad
import themachinethatgoesping.echosounders.simrad.datagrams as dg
import themachinethatgoesping.tools.progressbars as prg
import themachinethatgoesping.tools as tools
import themachinethatgoesping.navigation.nmea_0183 as NMEA
import numpy
import os, sys
from time import time
from tqdm import tqdm
from collections import defaultdict
import numpy as np

#load files
folders=[]
with open('test_data.txt','r') as ifi:
    for line in ifi:
        print("Folder:", line)
        folders.append(line)

if not len(folders):
    print("Please add test_data folders to 'test_data.txt'")

files = []
for folder in folders:
    for r,d,f in os.walk(folder):
        for file in f:
            if file.endswith('.raw'):
                files.append(r + '/' +file)
            
files.sort()
#files
file_name = files[0]

In [ ]:
fm = FileSimradRaw_mapped(files, show_progress=True)
print(fm)

In [ ]:
%%timeit -n1 -r1
A = {}
for nm in tqdm(fm.datagram_interface.datagrams("NME0")[:]):
    a=nm.decode()
    
    
    
    if nm.type == 'GGA' or nm.type == 'GLL' or nm.type == "RMC":
        a.get_latitude()
        a.get_longitude()
    
    continue
    
    if nm.type == 'ZDA':
        a = a.to_timestamp()
        pass
    
    if nm.type == 'RMC':
        pass
        
    if nm.type == 'GGA':
        pass
    if nm.type == 'GLL':
        pass
    if nm.type == 'HDT':
        pass
    if nm.type == 'GST':
        pass
    
    
        
    

In [ ]:
t1 = time()
simrad.test_speed_raw(fm,"NME0")
#print(len(fm)/(time()-t1),'it/s')
t2 = time()
simrad.test_speed_type(fm,"NME0")
#print(len(fm)/(time()-t2),'it/s')
t3 = time()
simrad.test_speed_raw(fm,"NME0")
#print(len(fm)/(time()-t3),'it/s')
t4 = time()
simrad.test_speed_decode_nmea(fm)
#print(len(fm)/(time()-t2),'it/s')
t5 = time()
print(round(100*(t4-t3)/(t5-t4),2))
print('raw :',len(fm.datagram_interface.datagrams('NME0'))/(t4-t3),'it/s')
print('read:',len(fm.datagram_interface.datagrams('NME0'))/(t3-t2),'it/s')
print('pack:',len(fm.datagram_interface.datagrams('NME0'))/(t5-t4),'it/s')

In [ ]:
from collections import defaultdict
packages = defaultdict(list)
for nmea in tqdm(fm.datagram_interface.datagrams("NME0")):
    t = nmea.name
    if t == "invalid":
        print(len(packages['invalid']),end='\r')
    else:
        t = nmea.sender + ' ' + nmea.type
        
    packages[t].append(nmea)
        
for k,v in packages.items():
    print(k,':',len(v))

In [ ]:
for k,v in packages.items():
    print(k,':',len(v))